In [1]:
import time
import json
import os

from easydict import EasyDict as edict
from imutils.paths import list_files
import requests as req
import redis

sep = os.path.sep

In [2]:
ROOT_PATH   = sep.join(os.getcwd().split(sep)[:-5])
CONFIG_PATH = f'{ROOT_PATH}/utils/configs/config.json'
CONFIG      = edict(json.loads(open(CONFIG_PATH, 'r').read()))
URLS        = {'appdetail' : 'https://store.steampowered.com/api/appdetails?appids' }

In [3]:
def return_or_print(url):
    
    res = req.get(url)
    if res.status_code == 200: return res.json()
    else: print(f'no response with code {res.status_code}')
    
    
def get_info(appid):
    
    res = return_or_print(f"{URLS['appdetail']}={appid}&l=korean")
    return res

In [4]:
key         = CONFIG.steam
host        = CONFIG.global_host
port        = CONFIG.redis_port
conn        = redis.StrictRedis(host = host, port = port, db = 2)

AttributeError: 'EasyDict' object has no attribute 'redis_port'

In [ ]:
apps = return_or_print('https://api.steampowered.com/ISteamApps/GetAppList/v2')['applist']['apps']

In [ ]:
sample = apps[150]['appid']
info   = get_info(sample)[str(sample)]['data']

In [ ]:
stime = time.time()

for idx, app in enumerate(apps[:3], 1):
    
    _id  = app['appid']
    
    try:
        info = get_info(_id)[str(_id)]['data']
        data = {
                    'name' : info['name'],
                    'data' : info
                }
        
        conn.set(f'id:{_id}', json.dumps(data))
    # except Exception as e: print(f'[{idx}]{_id} - {e}')
    except: pass
    
    finally: 

        time.sleep(1)
        
time.time() - stime